In [1]:
## Load libraries
# Interacting with the API
import json
from requests import get

# Working with arrays, tables
import numpy as np
import pandas as pd

# Working with datetime objects
import time
from time import sleep
import pytz
from datetime import datetime, date, timedelta

# Type Hinting for clarity
from typing import Callable

# Binary storage
import pyarrow as pa
import pyarrow.parquet as pq
# Report format
import openpyxl


# FUNCTION DEFINITIONS

def timeshift(report_startdate:int) -> int:
    """
        Shift the timestamp call to api by 15 minutes.
    """
    updatedtime = report_startdate + 900
    return(updatedtime)

def tsconverter(timest : int) -> int:
    """
        Convert the the date from the api to CST timezone to continue to pass it to the api.
    """
    tz = pytz.timezone("America/Chicago")
    dateversion = datetime.fromtimestamp(timest, tz = tz)
    loop_param_timestamp = int(pd.to_datetime(dateversion).timestamp())
    return(loop_param_timestamp)

def initradehist_ts(report_runtime:float, timez:str="America/Chicago") -> int:
    """
        Define a timestamp function that sets the timestamp parameter for Gemini API
        It is hardcoded for the past 24 hours from the run time.
    """
    tz = pytz.timezone(timez)
    start_time = report_runtime
    report_start = datetime.fromtimestamp(start_time, tz = tz)
    hrs24ago = report_start - timedelta(days=1)
    param_timestamp = int(pd.to_datetime(hrs24ago).timestamp())
    return(param_timestamp)
    

def call_tradehistory(symbolurl_struct:dict, key:str, timestamp:int) -> pd.DataFrame:
    """
        Function to call the Gemini Trade History API.
        symbolurl_struct: dictionary containing names of symbols in report.
        key: the symbol to get trade history for.
        timestamp: timestamp from which the trade history API will start to pull data from. 
            It will pull 500 points of data starting from this timestamp.
    """
    response_tradehistory = get(
            url = symbolurl_struct[key],
            params = {
                "since": timestamp,
                "limit_trades":500
                }
            )
    tradehistory_frame = pd.json_normalize(response_tradehistory.json())
    return (tradehistory_frame)

In [2]:

timez = "America/Chicago"
tz = pytz.timezone(timez)

# Set the report start date
report_runtime = datetime.now().replace(tzinfo=tz).timestamp()

# 15 minute intervals
report_interval = 900
# number of 15 minutes in 1 day 
max_calls = 1440/15/2
# Set the APIs we want to connect to:
tradehistory_baseurl = "https://api.gemini.com/v1/"

# Set trade history api endpoint
tradehistory_endpoint = "trades/"

# Set Symbols to be retreived
symbols = ["btcusd","ethbtc"]

# Build url string for each symbol
url_persymbol = ["".join([tradehistory_baseurl,tradehistory_endpoint,sym]) for sym in symbols]

# Create a symbol/url array structure for ongoing programmatic retreival
symbolurl_struct = {}
for symbol,url in zip(symbols,url_persymbol):
    symbolurl_struct[symbol] = url



### MAIN REPORT BUILDER

In [3]:
symbolurl_struct

{'btcusd': 'https://api.gemini.com/v1/trades/btcusd',
 'ethbtc': 'https://api.gemini.com/v1/trades/ethbtc'}

In [5]:
btcusd_response = get(
        url = symbolurl_struct["btcusd"],
        params = {
            "since": initradehist_ts(report_runtime = report_runtime, timez = timez),
            "limit_trades":500
            }
    )

# Turn the raw json response to a pandas dataframe
tradehistory_frame = pd.json_normalize(btcusd_response.json())
tradehistory_frame

report_frame = pd.DataFrame()
temp_apiframe = tradehistory_frame.copy()
api_ts = max(temp_apiframe["timestamp"])
converted_apits = tsconverter(api_ts)
boundar_l = converted_apits - 900*2
boundar_h = converted_apits + 900*2
stop_boundary = report_runtime - 900*2
apicall_count = 1

while boundar_h < stop_boundary or boundar_l < stop_boundary:
        print("\nGetting Trade History\n")
        
        sleep(1)
        shiftedts = timeshift(converted_apits)
        shiftedcheck = max(shiftedts,stop_boundary) == stop_boundary
        if shiftedcheck == False: 
            print(f"\nFinished calling trade history api before next call.\n")
            next_tscall = pd.Timestamp.fromtimestamp(shiftedcheck)
            print(f'Stop calling. Check date: {next_tscall}')
            break
        raw_tradehistory = call_tradehistory(symbolurl_struct,"btcusd", shiftedts)
        api_ts = max(raw_tradehistory["timestamp"])
        newconverted_apits = tsconverter(api_ts)
        print(f"\nMost recent date from trade history: {pd.Timestamp.fromtimestamp(newconverted_apits)}\n")
        boundar_l = newconverted_apits - 900*2
        boundar_h = newconverted_apits + 900*2
        
        if boundar_h > stop_boundary or boundar_l > stop_boundary:
            print(f"Finished calling trade history api.")
            print(pd.Timestamp.fromtimestamp(newconverted_apits))
            break
        
        else:
            report_frame = report_frame.append(raw_tradehistory, ignore_index=True)
            converted_apits = newconverted_apits

        apicall_count = apicall_count + 1
        print(f"\nApi call count: {apicall_count}\n")
        if apicall_count > max_calls:
            break


Getting Trade History



KeyboardInterrupt: 

In [17]:
date_check = datetime.fromtimestamp(report_frame['timestamp'].iloc[-1])
print()
csv_frame = report_frame.copy()

2021-03-18 10:02:30


In [51]:
def ts_convert(timest):
    dateversion = datetime.fromtimestamp(timest)
    return(dateversion)

def tsms_convert(time_ms):
    s = time_ms / 1000.0
    dttime = datetime.fromtimestamp(s)
    return dttime.strftime('%Y-%m-%d %H:%M:%S.%f')


print(csv_frame.head())
print("\n")

            timestamp    timestampms          tid     price      amount  \
0 2021-03-17 12:23:34  1616001814607  28558064578  55324.61  0.06253041   
1 2021-03-17 12:23:34  1616001814591  28558064479  55324.61  0.11437875   
2 2021-03-17 12:23:34  1616001814591  28558064477  55324.62  0.00562125   
3 2021-03-17 12:23:33  1616001813750  28558059771  55356.22      0.0001   
4 2021-03-17 12:23:33  1616001813750  28558059769  55356.22      0.0001   

  exchange  type  
0   gemini  sell  
1   gemini  sell  
2   gemini  sell  
3   gemini  sell  
4   gemini  sell  




### Converting the dates to a readable format for CSV 

In [ ]:
readable_timestamp = pd.Series(map(ts_convert,csv_frame['timestamp']))

In [ ]:
# CHECK
print(readable_timestamp[:-4])

In [ ]:
# Assign
csv_frame['timestamp'] = readable_timestamp

In [53]:
# Repeat for timestampms
readable_timestampms = pd.Series(map(tsms_convert, csv_frame['timestampms']))

0        2021-03-17 12:23:34.607000
1        2021-03-17 12:23:34.591000
2        2021-03-17 12:23:34.591000
3        2021-03-17 12:23:33.750000
4        2021-03-17 12:23:33.750000
                    ...            
19491    2021-03-18 10:02:49.205000
19492    2021-03-18 10:02:49.205000
19493    2021-03-18 10:02:46.805000
19494    2021-03-18 10:02:46.804000
19495    2021-03-18 10:02:46.804000
Length: 19496, dtype: object


,timestamp,timestampms,tid,price,amount,exchange,type
0,2021-03-17 12:23:34,2021-03-17 12:23:34.607000,28558064578,55324.61,0.06253041,gemini,sell
1,2021-03-17 12:23:34,2021-03-17 12:23:34.591000,28558064479,55324.61,0.11437875,gemini,sell
2,2021-03-17 12:23:34,2021-03-17 12:23:34.591000,28558064477,55324.62,0.00562125,gemini,sell
3,2021-03-17 12:23:33,2021-03-17 12:23:33.750000,28558059771,55356.22,0.0001,gemini,sell
4,2021-03-17 12:23:33,2021-03-17 12:23:33.750000,28558059769,55356.22,0.0001,gemini,sell


In [ ]:
print(readable_timestampms[:-4])

In [ ]:
csv_frame['timestampms'] = readable_timestampms

In [ ]:
csv_frame.head()

In [24]:
# If it looks okay, export to CSV
csv_frame.to_csv('tradehistory_daily_btcusd_gemini_210318.csv', index=False)

19496   2021-03-18 10:02:45
19497   2021-03-18 10:02:31
19498   2021-03-18 10:02:30
19499   2021-03-18 10:02:30
Name: timestamp, dtype: datetime64[ns]

### Save in a persistant binary format

In [7]:
pqtable = pa.Table.from_pandas(report_frame)
pq.write_table(pqtable, 'tradehistory_daily_btcusd_gemini_210318.parquet')

# Discussion

- Running again for ethbtc

- Cleaning up